In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
dim = (150, 150)
channel = (3, )
input_shape = dim + channel
batch_size = 16
epoch = 5

In [4]:
train_datagen = ImageDataGenerator(rescale = 1. / 255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1. / 255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

val_datagen = ImageDataGenerator(rescale = 1. / 255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

In [5]:
train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/dataset/train',
                                                    target_size = dim,
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True)

test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/dataset/test',
                                                    target_size = dim,
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True)

val_generator = val_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/dataset/validation',
                                                    target_size = dim,
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True)

num_class = test_generator.num_classes
labels = train_generator.class_indices.keys()

print(labels)

Found 3458 images belonging to 5 classes.
Found 432 images belonging to 5 classes.
Found 433 images belonging to 5 classes.
dict_keys(['daisy', 'dandelion', 'rose', 'sunflower', 'tulip'])


In [6]:
def tf_data_generator(generator, input_shape):
    num_class = generator.num_classes
    tf_generator = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types = (tf.float32, tf.float32),
        output_shapes = ([None,
                         input_shape[0],
                         input_shape[1],
                         input_shape[2]],
                        [None, num_class])
    )
    return tf_generator

In [7]:
train_data = tf_data_generator(train_generator, input_shape)
test_data = tf_data_generator(test_generator, input_shape)
val_data = tf_data_generator(val_generator, input_shape)

In [10]:
pip install -U --pre efficientnet

     |████████████████████████████████| 51kB 3.2MB/s 


In [11]:
from efficientnet.tfkeras import EfficientNetB1
from tensorflow.keras import layers, Sequential
from tensorflow.keras.models import Model

In [12]:
# base models
base_model = EfficientNetB1(
    input_shape = input_shape,
    include_top = False,
    weights = 'noisy-student',
    classes = num_class,
)

27017216/27010080 [==============================] - 1s 0us/step


In [13]:
# custom layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1024, activation = 'relu')(x)

predictions = layers.Dense(num_class, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [14]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 75, 75, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 75, 75, 32)   128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 75, 75, 32)   0           stem_bn[0][0]                    
_______________________________________________________________________________________

In [15]:
# Compile the model
print('Compiling Model...')
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

Compiling Model...


In [16]:
# Train Model
history = model.fit(x=train_data,
                   steps_per_epoch = len(train_generator),
                   epochs = epoch,
                   validation_data = val_data,
                   validation_steps = len(val_generator),
                   shuffle = True,
                   verbose = 1)

Epoch 1/5
217/217 [==============================] - 2651s 12s/step - loss: 0.7050 - accuracy: 0.7467 - val_loss: 0.4675 - val_accuracy: 0.8406
Epoch 2/5
217/217 [==============================] - 713s 3s/step - loss: 0.5046 - accuracy: 0.8175 - val_loss: 0.4619 - val_accuracy: 0.8591
Epoch 3/5
217/217 [==============================] - 714s 3s/step - loss: 0.4455 - accuracy: 0.8464 - val_loss: 0.4138 - val_accuracy: 0.8730
Epoch 4/5
217/217 [==============================] - 714s 3s/step - loss: 0.3452 - accuracy: 0.8783 - val_loss: 0.3808 - val_accuracy: 0.8891
Epoch 5/5
217/217 [==============================] - 717s 3s/step - loss: 0.3363 - accuracy: 0.8861 - val_loss: 0.4222 - val_accuracy: 0.8891


In [17]:
# Save Model
MODEL_BASE_PATH = "/content/drive/My Drive/Colab Notebooks/model"
PROJECT_NAME = "flowers"
SAVE_MODEL_NAME = "model5.h5"
save_model_path = os.path.join(MODEL_BASE_PATH, PROJECT_NAME, SAVE_MODEL_NAME)

if os.path.exists(os.path.join(MODEL_BASE_PATH, PROJECT_NAME)) == False:
    os.makedirs(os.path.join(MODEL_BASE_PATH, PROJECT_NAME))
    
print('Saving Model At {}...'.format(save_model_path))
model.save(save_model_path,include_optimizer=True)

Saving Model At /content/drive/My Drive/Colab Notebooks/model/flowers/model5.h5...


In [ ]:
# Evaluasi Model
# loss, acc = model.evaluate(test_data, steps = len(test_generator), verbose = 0)
# print('Accuracy pada data training: {:.4f} \nLoss pada data training: {:.4f}'.format(acc, loss), '\n')

# loss, acc = model.evaluate(test_data, steps = len(test_generator), verbose = 0)
# print('Accuracy pada data test: {:.4f} \nLoss pada data test: {:.4f}'.format(acc, loss), '\n')